Author: Vinícius Antoino Ramos Zecca

LinkedIn: /viniciusarz

Github: /ViniciusARZ

In [ ]:
import pandas as pd
import numpy as np
import sqlite3

As the challenge consists of table structures, I'm creating random integers as values to demonstrate if the query worked or not. *Ignore the values itself, they will not match conditions.*

In [ ]:
rng = np.random.default_rng()

transactions = pd.DataFrame(rng.integers(0, 100, size=(100, 4)), columns={'transaction_id','payment_profile_id','amount','payment_month'})

payment_profiles = pd.DataFrame(rng.integers(0, 100, size=(100, 5)), columns={'payment_profile_id','user_id','last_4_digits','address','plan_id'})

current_paid_users = pd.DataFrame(rng.integers(0, 100, size=(100, 3)), columns={'user_id','email','user_plan'})

Creating db

In [ ]:
try:
    conn = sqlite3.connect('Termly.db')
    cursor = conn.cursor()
    print("Database created and Successfully Connected to SQLite")

    sqlite_select_Query = "select sqlite_version();"
    cursor.execute(sqlite_select_Query)
    record = cursor.fetchall()
    print("SQLite Database Version is: ", record)
    cursor.close()

except sqlite3.Error as error:
    print("Error while connecting to sqlite", error)
finally:
    if conn:
        conn.commit()
        print("The SQLite connection is closed")

Database created and Successfully Connected to SQLite
SQLite Database Version is:  [('3.22.0',)]
The SQLite connection is closed


Creating tables inside db

In [ ]:
transactions.to_sql("transactions", conn, if_exists="replace")
payment_profiles.to_sql("payment_profiles", conn, if_exists="replace")
current_paid_users.to_sql("current_paid_users", conn, if_exists="replace")

Testing if it worked

In [ ]:
c = conn.cursor()
c.execute("SELECT * from transactions LIMIT 5")
results = c.fetchall()
for result in results:
  print(result)

(0, 38, 92, 0, 84)
(1, 37, 21, 57, 65)
(2, 11, 76, 82, 9)
(3, 58, 45, 92, 48)
(4, 44, 17, 9, 83)


Query to Solve: Return transaction_id and amount for all annual plan transactions.


In this case the query will not show anything because my values are Ints and my condition is a string.

In [ ]:
c = conn.cursor()
c.execute("SELECT user_plan, transactions.transaction_id, transactions.amount FROM current_paid_users LEFT JOIN payment_profiles ON current_paid_users.user_id = payment_profiles.user_id LEFT JOIN transactions ON payment_profiles.payment_profile_id = transactions.payment_profile_id WHERE user_plan = 'annual_plan'")
results = c.fetchall()
for result in results:
  print(result)

Query to Solve: Return the payment profile ID and sum of the amount paid by each user within
each month. Return data only for users who have more than 2 transactions per month.


In [ ]:
transactions.columns

Index(['payment_profile_id', 'transaction_id', 'amount', 'payment_month'], dtype='object')

In [ ]:
c = conn.cursor()
c.execute("SELECT payment_profile_id, SUM(amount) AS Total_Amount, COUNT(transaction_id) AS Total_Transactions, payment_month FROM transactions GROUP BY payment_Profile_id, payment_month HAVING Total_Transactions > 2 ORDER BY payment_month")
results = c.fetchall()
for result in results:
  print(result)

Query to Solve: Return all transactions greater than $10, sorted in descending order by month
and payment_profile_id.

In [ ]:
c = conn.cursor()
c.execute("SELECT * FROM transactions WHERE amount > 10 ORDER BY payment_month,payment_profile_id DESC LIMIT 5")
results = c.fetchall()
for result in results:
  print(result)

(22, 93, 78, 60, 0)
(97, 63, 70, 36, 1)
(38, 23, 89, 73, 2)
(80, 26, 73, 30, 3)
(78, 58, 56, 21, 4)


Query to Solve: Return the last 4 credit card digits and count of transactions for each payment
profile ID in the Payment_Profiles table. If there were no transactions, return 0

In [ ]:
payment_profiles.columns

Index(['address', 'last_4_digits', 'payment_profile_id', 'plan_id', 'user_id'], dtype='object')

In [ ]:
Transaction_Counts = pd.DataFrame(rng.integers(0, 100, size=(100, 2)), columns={'payment_profile_id','transaction_counts'})
Transaction_Counts.to_sql("transaction_counts", conn, if_exists="replace")
c = conn.cursor()
c.execute("SELECT last_4_digits, payment_profiles.payment_profile_id, CASE WHEN transaction_counts.transaction_counts = 0 THEN '0' ELSE transaction_counts.transaction_counts END AS Transactions FROM payment_profiles LEFT JOIN transaction_counts ON payment_profiles.payment_profile_id = transaction_counts.payment_profile_id LIMIT 5")
results = c.fetchall()
for result in results:
  print(result)

(60, 58, 8)
(60, 58, 62)
(96, 61, 10)
(96, 61, 52)
(96, 61, 89)


Query to Solve: Return all events in a unified table containing event IDs, user IDs, and event
types.

In [ ]:
Type_1_Events = pd.DataFrame(rng.integers(0, 100, size=(100, 4)), columns={'event_id','timestamp','user_id','event_type'})
Type_1_Events.to_sql("type_1_events", conn, if_exists="replace")
Type_2_Events = pd.DataFrame(rng.integers(0, 100, size=(100, 5)), columns={'event_id','email','notes','user_id','event_type'})
Type_2_Events.to_sql("type_2_events", conn, if_exists="replace")

c = conn.cursor()
c.execute("SELECT type_1_events.event_id, type_1_events.user_id, type_1_events.event_type FROM type_1_events UNION SELECT type_2_events.event_id, type_2_events.user_id, type_2_events.event_type FROM type_2_events LIMIT 5")
results = c.fetchall()
for result in results:
  print(result)

(0, 12, 42)
(0, 78, 17)
(0, 87, 17)
(1, 69, 87)
(2, 91, 4)


Query to Solve: Return the total amount, count of transactions and count of documents for
each payment profile ID in the Payment_Profiles table

In [ ]:
payment_profiles.columns

Index(['payment_profile_id', 'address', 'last_4_digits', 'user_id'], dtype='object')

In [ ]:
documents = pd.DataFrame(rng.integers(0, 100, size=(100, 2)), columns={'payment_profile_id','document_id'})
documents.to_sql("documents", conn, if_exists="replace")

c = conn.cursor()
c.execute("SELECT payment_profiles.payment_profile_id, SUM(transactions.amount) AS Total_Amount, COUNT(transactions.transaction_id) AS Total_Transactions, COUNT(documents.document_id) AS Total_Documents FROM payment_profiles LEFT JOIN transactions ON payment_profiles.payment_profile_id = transactions.payment_profile_id LEFT JOIN documents ON payment_profiles.payment_profile_id = documents.payment_profile_id GROUP BY payment_profiles.payment_profile_id LIMIT 5")
results = c.fetchall()
for result in results:
  print(result)

(2, 1075, 15, 15)
(4, 208, 4, 4)
(6, 276, 4, 4)
(7, 89, 2, 0)
(8, None, 0, 0)


Query to Solve: Return all transaction IDs and amounts along labeled according to the plan
type. Plan IDs of 1, 3, 5, and 7 should get the label “monthly_plan.” Plan IDs of 2, 4, 6, and 8
should get the label “annual_plan.”

In [ ]:
transactions.columns

Index(['payment_profile_id', 'transaction_id', 'amount', 'payment_month'], dtype='object')

In [ ]:
payment_profiles.columns

Index(['address', 'last_4_digits', 'payment_profile_id', 'plan_id', 'user_id'], dtype='object')

In [ ]:
c = conn.cursor()
c.execute("SELECT transactions.transaction_id, transactions.amount, CASE payment_profiles.plan_id WHEN 1 OR 3 OR 5 OR 7 THEN 'monthly_plan' WHEN 2 OR 4 OR 6 OR 8 THEN 'annual_plan' END AS Plans FROM transactions LEFT JOIN payment_profiles ON transactions.payment_profile_id = payment_profiles.payment_profile_id LIMIT 5")
results = c.fetchall()
for result in results:
  print(result)

(92, 0, None)
(21, 57, None)
(76, 82, None)
(45, 92, None)
(45, 92, None)
